### Importing Libraries

In [13]:
from dkube.sdk import *
import os

In [14]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client
from kfp import components
import json
from random import randint

### Initializing fields

In [15]:
user = os.getenv('USERNAME')
token = os.getenv('DKUBE_USER_ACCESS_TOKEN')
code_name = "sdk-mnist"
dataset_name = "sdk-mnist"
model_name = "sdk-mnist"
framework = "tensorflow_1.14"
image = "ocdr/d3-datascience-tf-cpu:v1.14-3"
training_script = "python model.py"
transformer_script = "sdk-examples/mnist-tf/transformer.py"

In [16]:
# Dkube Client handle
api = DkubeApi(token=token)

### Creating Repos using DKube SDK

In [17]:
print(f"Adding code {code_name}")
code = DkubeCode(user, name=code_name)
code.update_git_details("https://github.com/oneconvergence/dkube-examples-internal/tree/sdk-eg-mnist/sdk-examples/mnist-tf")
try:
    api.create_code(code)
    print(f"Code {code_name} added")
except BaseException as e:
    print(e)

print(f"Adding dataset {dataset_name}")
dataset = DkubeDataset(user, name=dataset_name)
try:
    api.create_dataset(dataset)
    print(f"Dataset {dataset_name} added")
except BaseException as e:
    print(e)

print(f"Adding model {model_name}")
model = DkubeModel(user, name=model_name)
model.update_model_source(source='dvs')
try:
    api.create_model(model)
    print(f"Model {model_name} added")
except BaseException as e:
    print(e)

Adding code sdk-mnist
{'code': 200, 'message': 'Added code sdk-mnist successfully', 'uuid': None}
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for completion, current state DOWNLOADING
code sdk-mnist - waiting for co

### Load Components

In [18]:

dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")


### Pipeline Definition

In [19]:

@dsl.pipeline(
  name='MNIST',
  description='A pipeline to train and serve the MNIST example.'
)
def mnist_pipeline(train_steps='200',
                   learning_rate='0.01',
                   batch_size='100',
                   token=token,code_name=code_name,dataset_name=dataset_name,model_name=model_name):
    
                   training = dkube_training_op(token, json.dumps({"image": image}),
                            framework="tensorflow", version="1.14",
                            program=code_name, run_script=training_script,
                            input_dataset_mounts='["tmp/data/"]',outputs='["sdk-mnist"]',
                            output_mounts='["/model"]')
                   
                   serving  = dkube_serving_op(token, training.outputs['artifact'],
                            device="cpu", serving_image=json.dumps({"image": "ocdr/tensorflowserver:1.14"}),
                            transformer_image=json.dumps({"image": image}),
                            transformer_project=code_name,
                            transformer_code=transformer_script).after(training).set_display_name("MNIST Serving")
         
                   

### Creating Experiment

In [20]:
experiment_name = 'MNIST pl'
client = kfp.Client(existing_token=token)
mnist_experiment = client.create_experiment(name=experiment_name)

### Compile and Upload the Pipeline

In [23]:
arguments = {"token":token}
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
try:
    pipeline = client.upload_pipeline("mnist-pipeline.zip", pipeline_name = "mnist-pipeline")
except BaseException as e:
    print(e)
runid = 1


### Create Run from pipeline

In [24]:
run = client.run_pipeline(mnist_experiment.id, job_name="[MNIST] Run" + str(runid), pipeline_id=pipeline.id, params=arguments)
runid += 1